**References**

[Tutorial: Working with Hugging Face Models and Datasets](https://github.com/anyuanay/medium/blob/main/src/working_huggingface/Working_with_HuggingFace_ch4_Fine_Tuning_Pretrained_Model_for_Question_Answering.ipynb)

[Fine-Tuning-for-Question-Answering-SQuAD-IndoQA](https://github.com/PrasetyoWidyantoro/Fine-Tuning-for-Question-Answering-SQuAD-IndoQA/blob/master/fine-tune-squad-dataset.ipynb)

# Import libraries

In [1]:
%pip install evaluate sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments, Trainer
from tqdm.auto import tqdm
import evaluate
import numpy as np
import os
import json
from sentence_transformers import SentenceTransformer, util

# Global variables

In [3]:
random_seed = 42

kaggle = True

model_name = "readerbench/RoGPT2-medium"
model_short_name = "RoGPT2-medium"

dataset_name = ("xquad", "xquad.ro")
dataset_short_name = "xquad"

# Import model and tokenizer

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForQuestionAnswering.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/563 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/869 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/985k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/542k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/357 [00:00<?, ?B/s]

# Split the dataset into train, validation and test

In [5]:
ds = load_dataset(dataset_name[0], dataset_name[1])

ds = ds['validation'].train_test_split(test_size=0.2)
ds_train = ds['train']
ds_test = ds['test']

ds_test = ds_test.train_test_split(test_size=0.5)
ds_val = ds_test['train']
ds_test = ds_test['test']

print(len(ds_train), len(ds_val), len(ds_test))

README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/244k [00:00<?, ?B/s]

Generating validation split:   0%|          | 0/1190 [00:00<?, ? examples/s]

952 119 119


# Process the dataset

## Train dataset

In [6]:
def standardize(text: str):
    return text
    # return text.replace("ţ", "ț").replace("ş", "ș").replace("Ţ", "Ț").replace("Ş", "Ș")

def process_and_tokenize(dataset, tokenizer) -> dict:
    # Standardize and prepare inputs
    questions = [q.strip() for q in dataset["question"]]
    contexts = dataset["context"]
    answers = dataset["answers"]

    # Tokenize with overflow handling and offset mapping
    inputs = tokenizer(
        questions,
        contexts,
        max_length=384,
        truncation="only_second",
        stride=128,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Create mappings
    sample_mapping = inputs["overflow_to_sample_mapping"]
    offset_mapping = inputs["offset_mapping"]

    # Initialize start and end positions
    start_positions = []
    end_positions = []

    for i, offsets in enumerate(offset_mapping):
        # Retrieve original sample index
        sample_idx = sample_mapping[i]

        # Get context answer data
        answer_texts = answers[sample_idx]["text"]
        answer_starts = answers[sample_idx]["answer_start"]

        # Choose the answer span to use
        # Here, prioritize the first listed answer for simplicity
        if len(answer_starts) > 1:
            start_char = answer_starts[1]
            end_char = start_char + len(answer_texts[1])
        else:
            start_char = answer_starts[0]
            end_char = start_char + len(answer_texts[0])

        # Token-level sequence IDs
        sequence_ids = inputs.sequence_ids(i)

        # Context span
        context_start = sequence_ids.index(1)
        context_end = next(
            (j - 1 for j in range(context_start, len(sequence_ids)) if sequence_ids[j] != 1),
            len(sequence_ids) - 1  # Default to the last valid token index
        )
        # Check if answer falls outside the current span
        if offsets[context_start][0] > start_char or offsets[context_end][1] < end_char:
            # Mark as no answer
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Find the token positions corresponding to the character-level answer
            start_token = next(
                (j for j in range(context_start, context_end + 1) if offsets[j][0] <= start_char < offsets[j][1]),
                None,
            )
            end_token = next(
                (j for j in range(context_end, context_start - 1, -1) if offsets[j][0] < end_char <= offsets[j][1]),
                None,
            )

            if start_token is not None and end_token is not None:
                start_positions.append(start_token)
                end_positions.append(end_token)
            else:
                # Fallback if token positions are not found
                start_positions.append(0)
                end_positions.append(0)

    # Update tokenized inputs with labels
    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions

    # Remove unnecessary fields
    inputs.pop("overflow_to_sample_mapping", None)
    inputs.pop("offset_mapping", None)

    return inputs


In [7]:
tokenizer.pad_token = tokenizer.eos_token

In [8]:
ds_tok_train = ds_train.map(lambda x: process_and_tokenize(dataset=x, tokenizer=tokenizer), batched=True, batch_size=64, remove_columns=ds_train.column_names)
 
print(len(ds_tok_train))

Map:   0%|          | 0/952 [00:00<?, ? examples/s]

1007


## Validation and test datasets

In [9]:
def process_and_tokenize_val_test(dataset: Dataset, tokenizer: AutoTokenizer) -> dict:
    # Extract from the dataset and standardize where possible
    questions = [standardize(q).strip() for q in dataset["question"]]
    contexts = [standardize(c) for c in dataset["context"]]
    
    inputs = tokenizer(
        questions,
        contexts,
        max_length=384,
        truncation="only_second",
        stride=128,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_mapping = inputs["overflow_to_sample_mapping"]
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_i = sample_mapping[i]
        example_ids.append(dataset["id"][sample_i])

        seq_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        
        for j, ofs in enumerate(offset):
            inputs['offset_mapping'][i][j] = ofs if seq_ids[j] == 1 else None

    inputs["example_id"] = example_ids
    
    inputs.pop("overflow_to_sample_mapping")
    
    return inputs

In [10]:
ds_tok_val = ds_val.map(lambda x: process_and_tokenize_val_test(dataset=x, tokenizer=tokenizer), batched=True, batch_size=64, remove_columns=ds_val.column_names)
ds_tok_test = ds_test.map(lambda x: process_and_tokenize_val_test(dataset=x, tokenizer=tokenizer), batched=True, batch_size=64, remove_columns=ds_test.column_names)
 
print(len(ds_tok_val), len(ds_tok_test))

Map:   0%|          | 0/119 [00:00<?, ? examples/s]

Map:   0%|          | 0/119 [00:00<?, ? examples/s]

126 124


# Make metrics

In [11]:
%pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=88b3c1deed049a5782cd1774a1ab7ddfbee714ee46af9dc3a5cf67b0f70b5e7b
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
Note: you may need to restart the kernel to use updated packages.


In [12]:
squad_metric = evaluate.load("squad")
bleu_metric = evaluate.load("bleu")
rouge_metric = evaluate.load("rouge")

In [13]:
def compute_metrics(start_logits: list, end_logits: list, tok: dict, dataset: Dataset, max_answer_length: int = 30, eval_answer: SentenceTransformer = SentenceTransformer("BlackKakapo/stsb-xlm-r-multilingual-ro")) -> dict:
    extracted_features = {}
    
    # Group extracted features by ids
    for i, feature in enumerate(tok):
        if feature["example_id"] not in extracted_features:
            extracted_features[feature["example_id"]] = [i]
        else:
            extracted_features[feature["example_id"]].append(i)
    
    golds_squad = [{"id": data["id"], "answers": data["answers"]} for data in dataset]
    preds_squad = []
    
    golds_bleu = []
    preds_bleu = []
    
    for data in tqdm(dataset):
        answers = []

        # Iterate over all the extracted features
        for i in extracted_features[data["id"]]:
            start_logit = start_logits[i]
            end_logit = end_logits[i]
            offs = tok[i]["offset_mapping"]

            # Get all combinations of start and end positions
            for start_i in range(len(start_logit)):
                for end_i in range(len(end_logit)):
                    # Continue on wrong answers
                    if offs[start_i] is None \
                        or offs[end_i] is None \
                        or end_i < start_i \
                        or end_i - start_i + 1 > max_answer_length:
                        continue
                    
                    # Add text and score
                    answer = {
                        "answer": data["context"][offs[start_i][0] : offs[end_i][1]],
                        "score": start_logit[start_i] + end_logit[end_i]
                    }
                    answers.append(answer)
                    
        preds_squad.append(
            {"id": data['id'], "prediction_text": max(answers, key=lambda x: x["score"])['answer']} 
                if len(answers) > 0 else 
                {"id": data['id'], "prediction_text": ""}
        )
        
        preds_bleu.append(
            max(answers, key=lambda x: x["score"])['answer']
                if len(answers) > 0 else 
                ""
        )
        
        max_i = 0
        max_len = len(data["answers"]["text"][max_i])
        for i in range(1, len(data["answers"]['text'])):
            if len(data["answers"]["text"][i]) > max_len:
                max_len = len(data["answers"]["text"][i])
                max_i = i
        
        golds_bleu.append(data["answers"]['text'][max_i])
        
    return {
        "squad": squad_metric.compute(predictions=preds_squad, references=golds_squad),
        "bleu": bleu_metric.compute(predictions=preds_bleu, references=golds_bleu),
        "rouge": rouge_metric.compute(predictions=preds_bleu, references=golds_bleu),
        "semantic_similarity": util.pytorch_cos_sim(eval_answer.encode(preds_bleu, convert_to_tensor=True), eval_answer.encode(golds_bleu, convert_to_tensor=True)).mean().item() * 100
    }

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.68k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/748 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/527 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# Training, evaluation and testing loop for hyperparameters

In [14]:
def output_metrics_to_file(metrics: dict, metric_type: str = None, lr: float = None, epoch: int = None):
    filename = os.path.join("/kaggle/working/" if kaggle else ".", "results", f"{model_short_name}-{dataset_short_name}-type_{metric_type}-lr_{lr}-epoch_{epoch:02}.json" if lr is not None and epoch is not None and metric_type is not None else f"{model_short_name}-{dataset_short_name}.json")
    with open(filename, "w") as f:
        json.dump(metrics, f, indent=4)
        
def save_model(trainer: Trainer, lr: float, epoch: int):
    trainer.save_model(os.path.join("/kaggle/working/" if kaggle else ".", "results", "models", f"{model_short_name}-{dataset_short_name}-{lr}-{epoch}"))
    
def load_model(lr: float, epoch: int) -> AutoModelForQuestionAnswering:
    return AutoModelForQuestionAnswering.from_pretrained(os.path.join("/kaggle/working/" if kaggle else ".", "results", "models", f"{model_short_name}-{dataset_short_name}-{lr}-{epoch}"))

## Get maximum answer length within 2 standard deviations from the mean of the training dataset

In [15]:
# mean_answer_length = np.mean([len(a["text"][0]) for a in ds_train["answers"]])
# std_dev_answer_length = np.std([len(a["text"][0]) for a in ds_train["answers"]])

# two_std_devs_above = mean_answer_length + 2 * std_dev_answer_length

In [16]:
all_models_eval = None

def tet_loop(lr_list: list, epochs: int, batch_size: int) -> None:
    global all_models_eval
    all_models_eval = {"validation": {lr:{epoch:[] for epoch in range(1, epochs + 1)} for lr in lr_list}, "test": {lr:{epoch:[] for epoch in range(1, epochs + 1)} for lr in lr_list}}
    
    eval_answer = SentenceTransformer("BlackKakapo/stsb-xlm-r-multilingual-ro")
    
    for lr in lr_list:
        model = AutoModelForQuestionAnswering.from_pretrained(model_name)

        warmup_steps = int(len(ds_tok_train) / batch_size * 1 / 10)
        args = TrainingArguments(
                output_dir="./results",
                eval_strategy="no",
                save_strategy="epoch",
                learning_rate=lr,
                num_train_epochs=1,
                weight_decay=0.01,
                per_device_train_batch_size=batch_size, 
                report_to="none",
                save_total_limit=1,
                warmup_steps=warmup_steps
            )
        trainer = Trainer(
            model=model,
            args=args,
            train_dataset=ds_tok_train,
            eval_dataset=ds_tok_val,
            tokenizer=tokenizer
        )
        
        for epoch in range(1, epochs + 1):
            print(f"Training with lr={lr} and epoch={epoch}")
            
            trainer.train()
            
            preds = trainer.predict(ds_tok_val)
            start_logits, end_logits = preds[0][0], preds[0][1]
            computed_metrics = compute_metrics(start_logits, end_logits, ds_tok_val, ds_val, 30, eval_answer)
            
            # output_metrics_to_file(computed_metrics, metric_type='validation', lr=lr, epoch=epoch)
            print(f"Validation for lr {lr} epoch {epoch}: ")
            # print(computed_metrics)
            print(computed_metrics['squad'])
            print(computed_metrics['semantic_similarity'])
            
            all_models_eval['validation'][lr][epoch] = computed_metrics
            
            preds = trainer.predict(ds_tok_test)
            start_logits2, end_logits2 = preds[0][0], preds[0][1]
            computed_metrics = compute_metrics(start_logits2, end_logits2, ds_tok_test, ds_test, 30, eval_answer)
            
            
            # output_metrics_to_file(computed_metrics, metric_type='test', lr=lr, epoch=epoch)
            all_models_eval['test'][lr][epoch] = computed_metrics
            

            print(f"Test for lr {lr} epoch {epoch}: ")
            print(computed_metrics['squad'])
            print(computed_metrics['semantic_similarity'])
            
            # save_model(trainer, lr, epoch)
            
        try:
            del trainer
            del model
            # device = cuda.get_current_device()
            # device.reset()
        except Exception as e:
            print(e)
            
            print(f"Finished training, validating and testing with lr={lr} and epochs={epochs}")

In [17]:
tet_loop(lr_list=[1e-4, 1e-3, 1e-5], epochs=16, batch_size=4)

pytorch_model.bin:   0%|          | 0.00/1.44G [00:00<?, ?B/s]

Some weights of GPT2ForQuestionAnswering were not initialized from the model checkpoint at readerbench/RoGPT2-medium and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-16-70356e6c222f>:25: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Training with lr=0.0001 and epoch=1


Step,Training Loss


  0%|          | 0/119 [00:00<?, ?it/s]

...

Validation for lr 1e-05 epoch 16: 
{'exact_match': 21.008403361344538, 'f1': 30.96476201029313}
17.062020301818848


  0%|          | 0/119 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Test for lr 1e-05 epoch 16: 
{'exact_match': 15.966386554621849, 'f1': 24.226832989288436}
17.827226221561432


In [18]:
output_metrics_to_file(all_models_eval)

In [19]:
!zip results_ml.zip results/*.json

  adding: results/RoGPT2-medium-xquad.json (deflated 85%)
